# **data_set do kaggle**

In [4]:
# IMPORTANTE: RODE ESTA CÉLULA PARA IMPORTAR SUAS FONTES DE DADOS DO KAGGLE,
# DEPOIS, SINTA-SE À VONTADE PARA DELETAR ESTA CÉLULA.
# NOTA: O AMBIENTE DESTE NOTEBOOK É DIFERENTE DO AMBIENTE PYTHON DO KAGGLE,
# PORTANTO, PODEM FALTAR BIBLIOTECAS USADAS PELO SEU NOTEBOOK.
import kagglehub
paultimothymooney_blood_cells_path = kagglehub.dataset_download('paultimothymooney/blood-cells')

print('Importação da fonte de dados concluída.')

Data source import complete.


# **Instalação de Dependências**

In [3]:
!pip install keras-cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 38.2 MB/s eta 0:00:00


# **Importação das Bibliotecas**

In [5]:
import keras_cv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import regularizers

Mounted at /content/drive


### funcoes

In [8]:
def transformar_imagem(imagem, rotulo):
    # tranforma a imagem com o data_augmentation, mas mantem o rotulo dela
    imagem = manipulacao_imagem(imagem, training=True)
    return imagem, rotulo

In [9]:
def para_one_hot(imagem, rotulo):
    # basicamente ele coloca uma etiqueta na imagem: exemplo: indice = 1 --> [0,1,0,0] para facilitar os calculos depois
    rotulo_one_hot = tf.one_hot(tf.cast(rotulo, tf.int32), depth=4)
    return imagem, rotulo_one_hot

# **Preparação e Otimização dos Dados**

### dataset

In [10]:
dataset_de_treino = tf.keras.utils.image_dataset_from_directory(
directory = "/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN", #pasta com os arquivos *aqui tem q mudar para o do drive, pq ele n consegue acessar o kaggle desse jeito, pelo menos não q eu saiba.
image_size=(128, 128), #redimenciona o tamanho da imagem. no caso, 128x128
batch_size=32, #define quantas imagens são carregadas por vez (lote), mais imagems por lotes = mais velocidade, menos imagem por lote = menos velocidade, exemplo batch 8 com 64 imagens = 8 lotes, batch 32 com 64 imagens = 2 lotes.
validation_split=0.2, # porcentagem reservada para validação, no caso 20%
subset="training", # indica que este conjunto é a parte de treino (e não validação)
seed=47, # define uma semente para garantir que a divisão entre treino/validação seja sempre igual
)
dataset_de_validacao = tf.keras.utils.image_dataset_from_directory(
directory = "/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN", #pasta com os arquivos *aqui tem q mudar para o do drive, pq ele n consegue acessar o kaggle desse jeito, pelo menos não q eu saiba.
image_size=(128, 128),
batch_size=32,
validation_split=0.2,
subset="validation", # indica que este conjunto é a parte de validação (e não treino)
seed=47,)

Found 9957 files belonging to 4 classes.
Using 7966 files for training.
Found 9957 files belonging to 4 classes.
Using 1991 files for validation.


### manipulação de imagem

In [11]:
# Define a sequência de transformações visuais que serão aplicadas às imagens (geralmente usa data_augmentation)
manipulacao_imagem = tf.keras.Sequential([
    # Normalização
    tf.keras.layers.Rescaling(1./255),

    # Espelhamento aleatório da imagem na horizontal e vertical
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),

    # Rotação aleatória
    tf.keras.layers.RandomRotation(0.2),

    # Zoom aleatório
    tf.keras.layers.RandomZoom(0.1),

    # Variação aleatória do contraste
    tf.keras.layers.RandomContrast(0.2),

    # Alteração aleatória do brilho
    tf.keras.layers.RandomBrightness(0.2),

    # Translação (deslocamento) aleatória horizontal e vertical
    tf.keras.layers.RandomTranslation(0.2, 0.2),

    # Adição de ruído gaussiano com desvio padrão de 0.1
    tf.keras.layers.GaussianNoise(0.1),

    # Corte aleatório de uma região da imagem (apaga uma pequena parte)
    keras_cv.layers.RandomCutout(height_factor=0.2, width_factor=0.2)
])


### aumentando o data_set e transformando em one-hot e aplicando otimização com o AUTOTUNE

In [12]:
# Filtra apenas as imagens da classe eosinófilo (rótulo 0)
eosinofilo_ds = dataset_de_treino.filter(lambda x, y: tf.reduce_any(tf.equal(y, 0)))
# Filtra apenas as imagens da classe monócito (rótulo 2)
monocito_ds = dataset_de_treino.filter(lambda x, y: tf.reduce_any(tf.equal(y, 2)))
# Filtra apenas as imagens da classe neutrófilo (rótulo 3)
neutrofilo_ds = dataset_de_treino.filter(lambda x, y: tf.reduce_any(tf.equal(y, 3)))

# Aplica transformações (data augmentation) nas imagens de eosinófilos e neutrófilos, a função esta la emcima
eosinofilo_modificado = eosinofilo_ds.map(transformar_imagem)
neutrofilo_modificado = neutrofilo_ds.map(transformar_imagem)
monocito_modificado = monocito_ds.map(transformar_imagem)

In [13]:
# Habilita otimização automática de performance para carregamento antecipado de dados
AUTOTUNE = tf.data.AUTOTUNE

# Concatena os dados aumentados ao dataset original de treino
dataset_de_treino_aumentado = dataset_de_treino.concatenate(eosinofilo_modificado)
dataset_de_treino_aumentado = dataset_de_treino.concatenate(neutrofilo_modificado)
dataset_de_treino_aumentado = dataset_de_treino.concatenate(monocito_modificado)

# Converte os rótulos inteiros para o formato one-hot
dataset_de_treino_final = dataset_de_treino_aumentado.map(para_one_hot)
dataset_de_validacao_final = dataset_de_validacao.map(para_one_hot)

# Otimizações de performance para o dataset de treino
dataset_de_treino_final = dataset_de_treino_final.cache() # armazena os dados na memória (mais rápido)
dataset_de_treino_final = dataset_de_treino_final.shuffle(buffer_size=1000, seed=47) # embaralha os dados
dataset_de_treino_final = dataset_de_treino_final.prefetch(buffer_size=AUTOTUNE) # pré-carrega lotes de forma paralela

# Pré-carrega o dataset de validação também
dataset_de_validacao_final = dataset_de_validacao_final.prefetch(buffer_size=AUTOTUNE)

# **Transfer machine**

### importando o MobileNetV2

In [27]:
tamanho_imagem = (128, 128, 3)  # Define o tamanho das imagens: 128x128 pixels e 3 canais de cor (RGB)

modelo_base = tf.keras.applications.MobileNetV2(
    input_shape=tamanho_imagem,   # Define o formato da entrada para o modelo base
    include_top=False,            # Remove a última camada (camada de classificação) do MobileNetV2, pq no nosso caso serao 4 saidas
    weights='imagenet'            # Carrega pesos pré-treinados no dataset ImageNet, geralmente os pesos iniciais sao mais genericos e os ultimos mais espeficicos
)

### Otimizando o MobileNetV2

In [28]:
# Obtém a saída do modelo base (MobileNetV2, sem a parte de classificação)
saida = modelo_base.output

# Aplica um Global Average Pooling para reduzir as dimensões espaciais
saida = tf.keras.layers.GlobalAveragePooling2D()(saida)

# Aplica Dropout para evitar overfitting(ficar mt com treino e ruim em validacao)
saida = tf.keras.layers.Dropout(0.5)(saida)

# Camada densa final com 4 neurônios (número de classes) para a predição
camada_predicao = tf.keras.layers.Dense(4, kernel_regularizer = tf.keras.regularizers.l2(0.001))(saida)

# Cria o modelo final, com entrada do modelo base e saída da camada de predição
modelo_final = tf.keras.Model(
    inputs=modelo_base.input,
    outputs=camada_predicao
)


### compilando

In [54]:
modelo_final.compile(
    loss=keras_cv.losses.FocalLoss(from_logits=True),         # Define a função de perda como Focal Loss (ajuda em problemas de classes desbalanceadas, no nosso caso ficou desbalanceado por eu ter concatenado la emcima)
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),   # Usa o otimizador Adam com taxa de aprendizado muito baixa
    metrics=['accuracy']                                      # Métrica para avaliar o desempenho: acurácia
)


# **Configuração dos Callbacks**

### caminho para salvar o melhor modelo

In [19]:
# Novo caminho: cria uma pasta 'meus_modelos' dentro do seu Google Drive
caminho_saida = '/content/drive/MyDrive/meus_modelos'

# Caminho completo do arquivo a ser salvo
caminho_arquivo_modelo = os.path.join(caminho_saida, 'melhor_modelo_transfer_colab.keras')

### funcoes

In [20]:
# Interrompe o treinamento se a validação parar de melhorar
parada = EarlyStopping(
    monitor='val_loss',           # Acompanha a perda (loss) no conjunto de validação
    patience=4,                   # Espera 4 épocas sem melhoria antes de interromper
    restore_best_weights=True     # Restaura os pesos do melhor ponto do treino (não os da última época)
)

# Reduz a taxa de aprendizado se a validação não melhorar
taxa_de_aprendizado = ReduceLROnPlateau(
    monitor='val_loss',           # Acompanha a perda de validação
    factor=0.5,                   # Reduz a taxa de aprendizado pela metade
    patience=2,                   # Espera 2 épocas sem melhoria para agir
    verbose=1                     # Mostra mensagem no console ao reduzir
)

# Salva automaticamente o melhor modelo durante o treinamento
salvar_melhor_modelo = ModelCheckpoint(
    caminho_arquivo_modelo,       # Caminho onde o modelo será salvo
    monitor='val_loss',           # Salva com base na melhor validação
    save_best_only=True,          # Salva apenas quando houver melhoria
    verbose=1                     # Exibe mensagem ao salvar
)

# **Treinamento do Modelo**

In [55]:
historico = modelo_final.fit(
    dataset_de_treino_final,                                        # Conjunto de dados de treino (com imagens e rótulos one-hot)
    validation_data=dataset_de_validacao_final,                     # Conjunto de validação para monitorar o desempenho
    epochs=10,                                                      # Número máximo de épocas para treinamento
    callbacks=[parada, taxa_de_aprendizado, salvar_melhor_modelo],  # Chamadas automáticas para controle do treino
)

Epoch 1/10
    496/Unknown 28s 26ms/step - accuracy: 0.7307 - loss: 0.1096
Epoch 1: val_loss did not improve from 0.09276
498/498 ━━━━━━━━━━━━━━━━━━━━ 34s 38ms/step - accuracy: 0.7307 - loss: 0.1096 - val_accuracy: 0.8292 - val_loss: 0.1021 - learning_rate: 1.0000e-05
Epoch 2/10
498/498 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7536 - loss: 0.0991
Epoch 2: val_loss did not improve from 0.09276
498/498 ━━━━━━━━━━━━━━━━━━━━ 26s 20ms/step - accuracy: 0.7536 - loss: 0.0991 - val_accuracy: 0.8443 - val_loss: 0.0974 - learning_rate: 1.0000e-05
Epoch 3/10
496/498 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7609 - loss: 0.0967
Epoch 3: val_loss did not improve from 0.09276
498/498 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.7608 - loss: 0.0968 - val_accuracy: 0.8247 - val_loss: 0.1053 - learning_rate: 1.0000e-05
Epoch 4/10
495/498 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7764 - loss: 0.0909
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch

# ajuste fino

In [53]:
modelo_final.trainable = True      # Permite que todas as camadas do modelo base sejam treináveis, em true a gente pode retreinar o modelo, perdendo os pesos q a gente carregou,para isso nao acontecer a gente congela alguns "neuronios"(abaixo)

print("Número de camadas no modelo base: ", len(modelo_base.layers))

ajuste_fino_a_partir = 30  # Congela as camadas até essa posição para ajuste fino

# Congela as camadas do modelo base para que não sejam treinadas
for layer in modelo_base.layers[:ajuste_fino_a_partir]:
    layer.trainable = False

'print("Número de camadas no modelo base: ", len(modelo_base.layers))\n\najuste_fino_a_partir = 120  # Congela as camadas até essa posição para ajuste fino\n\n# Congela as camadas do modelo base para que não sejam treinadas\nfor layer in modelo_base.layers[:ajuste_fino_a_partir]:\n    layer.trainable = False'

# **Visualização do Histórico de Treino**

In [ ]:
# Extrai os resultados do objeto 'historico'
acc = historico.history['accuracy']
val_acc = historico.history['val_accuracy']
loss = historico.history['loss']
val_loss = historico.history['val_loss']

# Define o alcance das épocas para o eixo X
epochs_range = range(len(acc))

# Cria a figura que conterá nossos dois gráficos
plt.figure(figsize=(15, 6))

# Gráfico 1: Acurácia de Treinamento vs. Validação
plt.subplot(1, 2, 1) # (1 linha, 2 colunas, primeiro gráfico)
plt.plot(epochs_range, acc, label='Acurácia de Treino')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Acurácia de Treinamento e Validação')

# Gráfico 2: Perda de Treinamento vs. Validação
plt.subplot(1, 2, 2) # (1 linha, 2 colunas, segundo gráfico)
plt.plot(epochs_range, loss, label='Perda de Treino')
plt.plot(epochs_range, val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.title('Perda de Treinamento e Validação')

# Exibe os gráficos
plt.show()